In [1]:
#%reload_ext autoreload
#%autoreload 2
import logging
log = lambda msg: logging.info(msg)
import os
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Building Day Pipeline

In [2]:
input_file = "C:\\dev\\play\\brainwave-data-day\\2024-11-15-09-10-50\\raw.fif"

In [3]:
import pandas as pd

input_dir = "C:\\dev\\play\\brainwave-data-day"
stats_df = pd.read_csv(input_dir + os.path.sep + "day_stats.csv")

In [4]:
import io
import contextlib
from datetime import timedelta
import convert 

# Load MNE
output_buffer = io.StringIO()
with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
    log("Loading MNE file " + input_file)
    raw, input_file_without_ext, mne_filtered = convert.load_mne_file(log, input_file)
    
    channels = raw.info['ch_names']
    sfreq = raw.info['sfreq']
    start_date = raw.info['meas_date']
    duration = timedelta(seconds=float(raw.times[-1]))
    end_date = start_date + duration
    
    log(f"Start date: {start_date} end {end_date} duration {duration} channels: {channels} sfreq: {sfreq}")


2024-11-16 11:32:10,436 - INFO - Loading MNE file C:\dev\play\brainwave-data-day\2024-11-15-09-10-50\raw.fif
2024-11-16 11:32:10,437 - INFO - Reading file C:\dev\play\brainwave-data-day\2024-11-15-09-10-50\raw.fif
2024-11-16 11:32:10,657 - INFO - Finished reading file C:\dev\play\brainwave-data-day\2024-11-15-09-10-50\raw.fif
2024-11-16 11:32:10,747 - INFO - Start date: 2024-11-15 09:10:50.479561+00:00 end 2024-11-15 09:22:06.719561+00:00 duration 0:11:16.240000 channels: ['Fpz-M1'] sfreq: 250.0


In [5]:
from memory import garbage_collect

# Save as EDF
garbage_collect(log)
log("Saving as EDF")
convert.save_mne_as_downsample_edf(log, mne_filtered, input_file_without_ext)


2024-11-16 11:32:10,819 - INFO - Memory Usage: 229.55 MB GC to 229.55 MB
2024-11-16 11:32:10,820 - INFO - Saving as EDF


Overwriting existing file.


C:\dev\play\brainwave-processor\convert.py:83: RuntimeWarning: EDF format requires equal-length data blocks, so 0.76 seconds of edge values were appended to all channels when writing the final block.
  mne.export.export_raw(input_file_without_ext + ".edf", resampled, overwrite=True)


In [6]:
import yasa_features

# YASA features
garbage_collect(log)
log("Extracting YASA features")
yasa_feats, channel_feats_dict = yasa_features.extract_yasa_features2(log, channels, mne_filtered)


2024-11-16 11:32:14,523 - INFO - Memory Usage: 279.14 MB GC to 279.14 MB
2024-11-16 11:32:14,524 - INFO - Extracting YASA features


In [7]:
# Assuming the index represents the epoch
df = yasa_feats
df['EpochTime'] = (df.index * 30) + start_date.timestamp()
df['TimestampUK'] = pd.to_datetime(df['EpochTime'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Europe/London')
df.drop('EpochTime', axis=1, inplace=True)

In [8]:
df

,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_alphaaa,Fpz-M1_eeg_alphaaa_c7min_norm,Fpz-M1_eeg_alphaaa_p2min_norm,Fpz-M1_eeg_alphaab,...,Fpz-M1_eeg_thetaabs,Fpz-M1_eeg_thetaabs_c7min_norm,Fpz-M1_eeg_thetaabs_p2min_norm,Fpz-M1_eeg_thetaabsaa,Fpz-M1_eeg_thetaabsaa_c7min_norm,Fpz-M1_eeg_thetaabsaa_p2min_norm,Fpz-M1_eeg_thetaabsab,Fpz-M1_eeg_thetaabsab_c7min_norm,Fpz-M1_eeg_thetaabsab_p2min_norm,TimestampUK
epoch,,,,,,,,,,,,,,,,,,,,,
0,7.072715e-09,0.155163,1.186594,0.012408,-0.441512,-0.259280,0.017858,-0.389062,-0.391071,1.002342,...,7.175257e-10,0.054778,0.987512,8.458806e-10,0.064853,1.020979,7.115116e-09,0.173322,1.223512,2024-11-15 09:10:50.479561090+00:00
1,2.076430e-09,0.059034,0.463656,0.017996,-0.463431,0.416601,0.026784,-0.373252,0.323594,0.968025,...,2.617815e-10,-0.014426,0.444451,3.184976e-10,-0.010554,0.465848,2.011739e-09,0.066354,0.474705,2024-11-15 09:11:20.479561090+00:00
2,4.383291e-09,0.057731,0.445204,0.006967,-0.537483,-0.247380,0.010442,-0.426132,-0.310482,0.971955,...,3.544331e-10,-0.002489,0.337032,4.015122e-10,-0.002658,0.339059,4.350651e-09,0.058801,0.453892,2024-11-15 09:11:50.479561090+00:00
3,1.534609e-09,0.027714,0.229883,0.014031,-0.403262,-0.152211,0.022444,-0.343588,-0.147027,0.994820,...,1.014617e-10,0.002489,0.132603,1.349024e-10,0.002658,0.135346,1.461324e-09,0.022223,0.231513,2024-11-15 09:12:20.479561090+00:00
4,3.456014e-09,0.033122,-0.031776,0.011661,-0.262787,-0.197386,0.016679,-0.278007,-0.194254,0.969302,...,2.575200e-10,0.048728,-0.141467,3.153370e-10,0.048148,-0.143883,3.319718e-09,0.023593,-0.046932,2024-11-15 09:12:50.479561090+00:00
5,1.229371e-09,0.034908,-0.093059,0.016853,-0.032513,-0.266527,0.025717,-0.156359,-0.236944,0.959808,...,1.424436e-10,0.110981,-0.212568,1.736859e-10,0.109323,-0.220099,1.145530e-09,0.023266,-0.110480,2024-11-15 09:13:20.479561090+00:00
6,1.208887e-09,0.074112,-0.322719,0.014040,0.070771,0.161199,0.025599,-0.111443,0.369859,1.034752,...,1.093356e-10,0.194074,-0.358596,1.387190e-10,0.191623,-0.358409,1.171592e-09,0.062363,-0.343708,2024-11-15 09:13:50.479561090+00:00
7,6.455377e-09,0.151462,0.033286,0.009201,0.185684,-0.130869,0.013683,-0.111680,0.019119,0.996009,...,7.364498e-10,0.314209,0.019728,8.231722e-10,0.310961,0.003833,6.254216e-09,0.140155,0.007917,2024-11-15 09:14:20.479561090+00:00
8,6.425787e-09,0.169624,0.248142,0.012039,0.343572,-0.107969,0.017416,-0.077937,0.048654,0.953363,...,6.852364e-10,0.363259,0.274560,8.010894e-10,0.363698,0.259488,6.261685e-09,0.160589,0.223751,2024-11-15 09:14:50.479561090+00:00


In [9]:
import scaling

# Scaled
scale_by_stats = scaling.scale_by_stats(df, stats_df)
df = df.join(scale_by_stats.add_suffix('_s'))


In [10]:
df

,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_alphaaa,Fpz-M1_eeg_alphaaa_c7min_norm,Fpz-M1_eeg_alphaaa_p2min_norm,Fpz-M1_eeg_alphaab,...,Fpz-M1_eeg_thetaab_p2min_norm_s,Fpz-M1_eeg_thetaabs_s,Fpz-M1_eeg_thetaabs_c7min_norm_s,Fpz-M1_eeg_thetaabs_p2min_norm_s,Fpz-M1_eeg_thetaabsaa_s,Fpz-M1_eeg_thetaabsaa_c7min_norm_s,Fpz-M1_eeg_thetaabsaa_p2min_norm_s,Fpz-M1_eeg_thetaabsab_s,Fpz-M1_eeg_thetaabsab_c7min_norm_s,Fpz-M1_eeg_thetaabsab_p2min_norm_s
epoch,,,,,,,,,,,,,,,,,,,,,
0,7.072715e-09,0.155163,1.186594,0.012408,-0.441512,-0.259280,0.017858,-0.389062,-0.391071,1.002342,...,1.145321,1.635546,0.681690,1.702264,1.661905,0.714017,1.767508,1.611073,0.869115,2.091831
1,2.076430e-09,0.059034,0.463656,0.017996,-0.463431,0.416601,0.026784,-0.373252,0.323594,0.968025,...,0.462729,0.503070,0.575787,1.014822,0.512241,0.584579,1.040598,0.365217,0.692284,1.056047
2,4.383291e-09,0.057731,0.445204,0.006967,-0.537483,-0.247380,0.010442,-0.426132,-0.310482,0.971955,...,0.405784,0.733299,0.594053,0.878845,0.693208,0.598132,0.874576,0.936201,0.679798,1.027257
3,1.534609e-09,0.027714,0.229883,0.014031,-0.403262,-0.152211,0.022444,-0.343588,-0.147027,0.994820,...,0.512454,0.104692,0.601672,0.620066,0.112014,0.607256,0.607826,0.230848,0.619330,0.719652
4,3.456014e-09,0.033122,-0.031776,0.011661,-0.262787,-0.197386,0.016679,-0.278007,-0.194254,0.969302,...,0.236268,0.492480,0.672431,0.273131,0.505351,0.685343,0.242192,0.684526,0.621596,0.334494
5,1.229371e-09,0.034908,-0.093059,0.016853,-0.032513,-0.266527,0.025717,-0.156359,-0.236944,0.959808,...,0.175766,0.206528,0.767698,0.183127,0.196560,0.790351,0.142393,0.153755,0.621056,0.246591
6,1.208887e-09,0.074112,-0.322719,0.014040,0.070771,0.161199,0.025599,-0.111443,0.369859,1.034752,...,0.652353,0.124258,0.894856,-0.001725,0.120334,0.931624,-0.038716,0.160117,0.685687,-0.076021
7,6.455377e-09,0.151462,0.033286,0.009201,0.185684,-0.130869,0.013683,-0.111680,0.019119,0.996009,...,0.703826,1.682570,1.078701,0.477181,1.612402,1.136472,0.435617,1.400907,0.814286,0.410363
8,6.425787e-09,0.169624,0.248142,0.012039,0.343572,-0.107969,0.017416,-0.077937,0.048654,0.953363,...,0.564264,1.555310,1.153763,0.799764,1.564263,1.226997,0.770382,1.402730,0.848066,0.708915


In [11]:

# Main channel
df['Source'] = channels[0]
df = df.copy()
scaling.add_main_channel(df)


C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[renamed] = df[col]
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[renamed] = df[col]
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

In [12]:
df

,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_alphaaa,Fpz-M1_eeg_alphaaa_c7min_norm,Fpz-M1_eeg_alphaaa_p2min_norm,Fpz-M1_eeg_alphaab,...,Main_eeg_thetaab_p2min_norm_s,Main_eeg_thetaabs_s,Main_eeg_thetaabs_c7min_norm_s,Main_eeg_thetaabs_p2min_norm_s,Main_eeg_thetaabsaa_s,Main_eeg_thetaabsaa_c7min_norm_s,Main_eeg_thetaabsaa_p2min_norm_s,Main_eeg_thetaabsab_s,Main_eeg_thetaabsab_c7min_norm_s,Main_eeg_thetaabsab_p2min_norm_s
epoch,,,,,,,,,,,,,,,,,,,,,
0,7.072715e-09,0.155163,1.186594,0.012408,-0.441512,-0.259280,0.017858,-0.389062,-0.391071,1.002342,...,1.145321,1.635546,0.681690,1.702264,1.661905,0.714017,1.767508,1.611073,0.869115,2.091831
1,2.076430e-09,0.059034,0.463656,0.017996,-0.463431,0.416601,0.026784,-0.373252,0.323594,0.968025,...,0.462729,0.503070,0.575787,1.014822,0.512241,0.584579,1.040598,0.365217,0.692284,1.056047
2,4.383291e-09,0.057731,0.445204,0.006967,-0.537483,-0.247380,0.010442,-0.426132,-0.310482,0.971955,...,0.405784,0.733299,0.594053,0.878845,0.693208,0.598132,0.874576,0.936201,0.679798,1.027257
3,1.534609e-09,0.027714,0.229883,0.014031,-0.403262,-0.152211,0.022444,-0.343588,-0.147027,0.994820,...,0.512454,0.104692,0.601672,0.620066,0.112014,0.607256,0.607826,0.230848,0.619330,0.719652
4,3.456014e-09,0.033122,-0.031776,0.011661,-0.262787,-0.197386,0.016679,-0.278007,-0.194254,0.969302,...,0.236268,0.492480,0.672431,0.273131,0.505351,0.685343,0.242192,0.684526,0.621596,0.334494
5,1.229371e-09,0.034908,-0.093059,0.016853,-0.032513,-0.266527,0.025717,-0.156359,-0.236944,0.959808,...,0.175766,0.206528,0.767698,0.183127,0.196560,0.790351,0.142393,0.153755,0.621056,0.246591
6,1.208887e-09,0.074112,-0.322719,0.014040,0.070771,0.161199,0.025599,-0.111443,0.369859,1.034752,...,0.652353,0.124258,0.894856,-0.001725,0.120334,0.931624,-0.038716,0.160117,0.685687,-0.076021
7,6.455377e-09,0.151462,0.033286,0.009201,0.185684,-0.130869,0.013683,-0.111680,0.019119,0.996009,...,0.703826,1.682570,1.078701,0.477181,1.612402,1.136472,0.435617,1.400907,0.814286,0.410363
8,6.425787e-09,0.169624,0.248142,0.012039,0.343572,-0.107969,0.017416,-0.077937,0.048654,0.953363,...,0.564264,1.555310,1.153763,0.799764,1.564263,1.226997,0.770382,1.402730,0.848066,0.708915


In [13]:
columns = df.columns.tolist()
columns.insert(0, columns.pop(columns.index('TimestampUK')))
df = df.reindex(columns=columns)

In [14]:
columns

['TimestampUK',
 'Fpz-M1_eeg_abspow',
 'Fpz-M1_eeg_abspow_c7min_norm',
 'Fpz-M1_eeg_abspow_p2min_norm',
 'Fpz-M1_eeg_alpha',
 'Fpz-M1_eeg_alpha_c7min_norm',
 'Fpz-M1_eeg_alpha_p2min_norm',
 'Fpz-M1_eeg_alphaaa',
 'Fpz-M1_eeg_alphaaa_c7min_norm',
 'Fpz-M1_eeg_alphaaa_p2min_norm',
 'Fpz-M1_eeg_alphaab',
 'Fpz-M1_eeg_alphaab_c7min_norm',
 'Fpz-M1_eeg_alphaab_p2min_norm',
 'Fpz-M1_eeg_alphaabs',
 'Fpz-M1_eeg_alphaabs_c7min_norm',
 'Fpz-M1_eeg_alphaabs_p2min_norm',
 'Fpz-M1_eeg_alphaabsaa',
 'Fpz-M1_eeg_alphaabsaa_c7min_norm',
 'Fpz-M1_eeg_alphaabsaa_p2min_norm',
 'Fpz-M1_eeg_alphaabsab',
 'Fpz-M1_eeg_alphaabsab_c7min_norm',
 'Fpz-M1_eeg_alphaabsab_p2min_norm',
 'Fpz-M1_eeg_at',
 'Fpz-M1_eeg_at_c7min_norm',
 'Fpz-M1_eeg_at_p2min_norm',
 'Fpz-M1_eeg_auc',
 'Fpz-M1_eeg_auc_c7min_norm',
 'Fpz-M1_eeg_auc_p2min_norm',
 'Fpz-M1_eeg_beta',
 'Fpz-M1_eeg_beta_c7min_norm',
 'Fpz-M1_eeg_beta_p2min_norm',
 'Fpz-M1_eeg_betaabs',
 'Fpz-M1_eeg_betaabs_c7min_norm',
 'Fpz-M1_eeg_betaabs_p2min_norm',
 'Fpz-M

In [15]:
df

,TimestampUK,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_alphaaa,Fpz-M1_eeg_alphaaa_c7min_norm,Fpz-M1_eeg_alphaaa_p2min_norm,...,Main_eeg_thetaab_p2min_norm_s,Main_eeg_thetaabs_s,Main_eeg_thetaabs_c7min_norm_s,Main_eeg_thetaabs_p2min_norm_s,Main_eeg_thetaabsaa_s,Main_eeg_thetaabsaa_c7min_norm_s,Main_eeg_thetaabsaa_p2min_norm_s,Main_eeg_thetaabsab_s,Main_eeg_thetaabsab_c7min_norm_s,Main_eeg_thetaabsab_p2min_norm_s
epoch,,,,,,,,,,,,,,,,,,,,,
0,2024-11-15 09:10:50.479561090+00:00,7.072715e-09,0.155163,1.186594,0.012408,-0.441512,-0.259280,0.017858,-0.389062,-0.391071,...,1.145321,1.635546,0.681690,1.702264,1.661905,0.714017,1.767508,1.611073,0.869115,2.091831
1,2024-11-15 09:11:20.479561090+00:00,2.076430e-09,0.059034,0.463656,0.017996,-0.463431,0.416601,0.026784,-0.373252,0.323594,...,0.462729,0.503070,0.575787,1.014822,0.512241,0.584579,1.040598,0.365217,0.692284,1.056047
2,2024-11-15 09:11:50.479561090+00:00,4.383291e-09,0.057731,0.445204,0.006967,-0.537483,-0.247380,0.010442,-0.426132,-0.310482,...,0.405784,0.733299,0.594053,0.878845,0.693208,0.598132,0.874576,0.936201,0.679798,1.027257
3,2024-11-15 09:12:20.479561090+00:00,1.534609e-09,0.027714,0.229883,0.014031,-0.403262,-0.152211,0.022444,-0.343588,-0.147027,...,0.512454,0.104692,0.601672,0.620066,0.112014,0.607256,0.607826,0.230848,0.619330,0.719652
4,2024-11-15 09:12:50.479561090+00:00,3.456014e-09,0.033122,-0.031776,0.011661,-0.262787,-0.197386,0.016679,-0.278007,-0.194254,...,0.236268,0.492480,0.672431,0.273131,0.505351,0.685343,0.242192,0.684526,0.621596,0.334494
5,2024-11-15 09:13:20.479561090+00:00,1.229371e-09,0.034908,-0.093059,0.016853,-0.032513,-0.266527,0.025717,-0.156359,-0.236944,...,0.175766,0.206528,0.767698,0.183127,0.196560,0.790351,0.142393,0.153755,0.621056,0.246591
6,2024-11-15 09:13:50.479561090+00:00,1.208887e-09,0.074112,-0.322719,0.014040,0.070771,0.161199,0.025599,-0.111443,0.369859,...,0.652353,0.124258,0.894856,-0.001725,0.120334,0.931624,-0.038716,0.160117,0.685687,-0.076021
7,2024-11-15 09:14:20.479561090+00:00,6.455377e-09,0.151462,0.033286,0.009201,0.185684,-0.130869,0.013683,-0.111680,0.019119,...,0.703826,1.682570,1.078701,0.477181,1.612402,1.136472,0.435617,1.400907,0.814286,0.410363
8,2024-11-15 09:14:50.479561090+00:00,6.425787e-09,0.169624,0.248142,0.012039,0.343572,-0.107969,0.017416,-0.077937,0.048654,...,0.564264,1.555310,1.153763,0.799764,1.564263,1.226997,0.770382,1.402730,0.848066,0.708915


In [16]:
df['TimestampUK']

epoch
0    2024-11-15 09:10:50.479561090+00:00
1    2024-11-15 09:11:20.479561090+00:00
2    2024-11-15 09:11:50.479561090+00:00
3    2024-11-15 09:12:20.479561090+00:00
4    2024-11-15 09:12:50.479561090+00:00
5    2024-11-15 09:13:20.479561090+00:00
6    2024-11-15 09:13:50.479561090+00:00
7    2024-11-15 09:14:20.479561090+00:00
8    2024-11-15 09:14:50.479561090+00:00
9    2024-11-15 09:15:20.479561090+00:00
10   2024-11-15 09:15:50.479561090+00:00
11   2024-11-15 09:16:20.479561090+00:00
12   2024-11-15 09:16:50.479561090+00:00
13   2024-11-15 09:17:20.479561090+00:00
14   2024-11-15 09:17:50.479561090+00:00
15   2024-11-15 09:18:20.479561090+00:00
16   2024-11-15 09:18:50.479561090+00:00
17   2024-11-15 09:19:20.479561090+00:00
18   2024-11-15 09:19:50.479561090+00:00
19   2024-11-15 09:20:20.479561090+00:00
20   2024-11-15 09:20:50.479561090+00:00
21   2024-11-15 09:21:20.479561090+00:00
Name: TimestampUK, dtype: datetime64[ns, Europe/London]

In [17]:
df
# Remvoe when YASA-esque model restored
output_csv_file = input_file_without_ext + ".output.csv"
df.to_csv(output_csv_file)

log("All done! " + input_file)


2024-11-16 11:32:16,885 - INFO - All done! C:\dev\play\brainwave-data-day\2024-11-15-09-10-50\raw.fif


In [18]:
from catboost import CatBoostClassifier
from models.eeg_states.eeg_states_model import predict_only_day_energy_model_pipeline

# DayEnergy model
models_and_data = [predict_only_day_energy_model_pipeline('ignored', df, False)]
model = CatBoostClassifier()
model.load_model("models/day_data/day-energy-non-realtime_catboost_model.cbm")
predictions = model.predict_proba(models_and_data[0].X)
predictions_df = pd.DataFrame(predictions, index=models_and_data[0].X.index)
predictions_df['DayEnergy'] = predictions_df[1]
predictions_df.drop([0, 1], axis=1, inplace=True)
df = pd.concat([df, predictions_df], axis=1)


['Main_eeg_abspow_s', 'Main_eeg_abspow_c7min_norm_s', 'Main_eeg_abspow_p2min_norm_s', 'Main_eeg_alpha_s', 'Main_eeg_alpha_c7min_norm_s', 'Main_eeg_alpha_p2min_norm_s', 'Main_eeg_alphaaa_s', 'Main_eeg_alphaaa_c7min_norm_s', 'Main_eeg_alphaaa_p2min_norm_s', 'Main_eeg_alphaab_s', 'Main_eeg_alphaab_c7min_norm_s', 'Main_eeg_alphaab_p2min_norm_s', 'Main_eeg_alphaabs_s', 'Main_eeg_alphaabs_c7min_norm_s', 'Main_eeg_alphaabs_p2min_norm_s', 'Main_eeg_alphaabsaa_s', 'Main_eeg_alphaabsaa_c7min_norm_s', 'Main_eeg_alphaabsaa_p2min_norm_s', 'Main_eeg_alphaabsab_s', 'Main_eeg_alphaabsab_c7min_norm_s', 'Main_eeg_alphaabsab_p2min_norm_s', 'Main_eeg_at_s', 'Main_eeg_at_c7min_norm_s', 'Main_eeg_at_p2min_norm_s', 'Main_eeg_auc_s', 'Main_eeg_auc_c7min_norm_s', 'Main_eeg_auc_p2min_norm_s', 'Main_eeg_beta_s', 'Main_eeg_beta_c7min_norm_s', 'Main_eeg_beta_p2min_norm_s', 'Main_eeg_betaabs_s', 'Main_eeg_betaabs_c7min_norm_s', 'Main_eeg_betaabs_p2min_norm_s', 'Main_eeg_db_s', 'Main_eeg_db_c7min_norm_s', 'Main_eeg_

In [19]:
df[['TimestampUK', 'DayEnergy']]

,TimestampUK,DayEnergy
epoch,,
0,2024-11-15 09:10:50.479561090+00:00,0.572675
1,2024-11-15 09:11:20.479561090+00:00,0.572705
2,2024-11-15 09:11:50.479561090+00:00,0.572769
3,2024-11-15 09:12:20.479561090+00:00,0.573382
4,2024-11-15 09:12:50.479561090+00:00,0.573306
5,2024-11-15 09:13:20.479561090+00:00,0.571491
6,2024-11-15 09:13:50.479561090+00:00,0.573147
7,2024-11-15 09:14:20.479561090+00:00,0.574245
8,2024-11-15 09:14:50.479561090+00:00,0.574252
